## 1. mongodb 문법 실습

* cine21 배우 랭킹 사이트 크롤링
  - 사이트 주소: http://www.cine21.com/rank/person/
  - 요청 방식 확인 방법: 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form data
      - section = 'actor'
      - period_start = '2018-08'
      - gender = 'all'
      - page = 3 

#### 배우 랭킹 및 상세 정보 크롤링 및 mongodb 저장 (full source)

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list)

#### mongodb 접속 기본 코드

In [2]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print (doc)
actor = actor_collection

{'_id': ObjectId('61122bbaff5c0ecb955459d4'), '배우이름': '하정우', '흥행지수': 23123, '출연영화': ['신과 함께-인과 연', '백두산', 'PMC: 더 벙커', '클로젯', '신과 함께-죄와 벌', '멋진 하루'], '랭킹': '1', '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}
{'_id': ObjectId('61122bbaff5c0ecb955459d5'), '배우이름': '마동석', '흥행지수': 20710, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '다른 이름': 'Ma Dongseok', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg'}
{'_id': ObjectId('61122bbaff5c0ecb955459d6'), '배우이름': '이병헌', '흥행지수': 17424, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': '3', '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 7

### 1.1. 컬럼명 변경 
* 저장되 있는 mongodb 데이터의 컬럼명을 변경하는 방법
* update_one()/update_many() 함수 활용

In [3]:
actor_collection.update_many ( {}, {'$rename': {'다른 이름':'다른이름'}} )

In [4]:
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d4'), '배우이름': '하정우', '흥행지수': 23123, '출연영화': ['신과 함께-인과 연', '백두산', 'PMC: 더 벙커', '클로젯', '신과 함께-죄와 벌', '멋진 하루'], '랭킹': '1', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('61122bbaff5c0ecb955459d5'), '배우이름': '마동석', '흥행지수': 20710, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}
{'_id': ObjectId('61122bbaff5c0ecb955459d6'), '배우이름': '이병헌', '흥행지수': 17424, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': '3', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모

#### 참고: collection 변수명 변경

In [5]:
actor = actor_collection

### 1.2 find의 다양한 문법 - sort

In [6]:
docs = actor_collection.find({}).sort('생년월일', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb95545a44'), '배우이름': '정현준', '흥행지수': 2469, '출연영화': ['기생충', '이웃사촌'], '랭킹': '113', '직업': '배우', '생년월일': '2011-11-08', '성별': '남'}
{'_id': ObjectId('61122bbaff5c0ecb95545a2d'), '배우이름': '이레', '흥행지수': 3123, '출연영화': ['증인', '반도', '너의 이름은.', '개를 훔치는 완벽한 방법', '소원'], '랭킹': '90', '직업': '배우', '생년월일': '2006-03-12', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/leere.vv/\n'}
{'_id': ObjectId('61122bbaff5c0ecb95545a20'), '배우이름': '이재인', '흥행지수': 3639, '출연영화': ['봉오동 전투', '사바하', '발신제한', '어른도감', '아워 바디', '아이 캔 스피크'], '랭킹': '77', '직업': '배우', '생년월일': '2004-02-06', '성별': '여'}
{'_id': ObjectId('61122bbaff5c0ecb955459e2'), '배우이름': '김향기', '흥행지수': 9308, '출연영화': ['신과 함께-인과 연', '아이', '영주', '증인', '신과 함께-죄와 벌', '우아한 거짓말'], '랭킹': '15', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': '\nhttps://twitter.com/smell2001\n', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('61122bbaff5c0ecb95545a47'), '배우이름': '성유빈', '흥행지수': 2385, '출연영화': ['봉오동 전투', '윤희에게', '생일', '아이 캔 스피크'], '랭킹': '116', '직업': '

### 1.3 find의 다양한 문법 - exists

In [7]:
docs = actor.find({'특기': {'$exists':True}}).sort('흥행지수').limit(5)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb95545a5e'), '배우이름': '류승범', '흥행지수': 1899, '출연영화': ['타짜: 원 아이드 잭', '죽거나 혹은 나쁘거나', '아치와 씨팍', '와이키키 브라더스', '복수는 나의 것', '부당거래'], '랭킹': '139', '직업': '배우', '생년월일': '1980-08-09', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '대동상업고등학교 중퇴', '취미': '영화보기, 스노우보드', '특기': '디제잉', '소속사': '웰메이드스타엠'}
{'_id': ObjectId('61122bbaff5c0ecb95545a5d'), '배우이름': '송하윤', '흥행지수': 1936, '출연영화': ['완벽한 타인', '제보자'], '랭킹': '138', '직업': '배우', '생년월일': '1986-12-02', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/hayoonsong1202/\n', '신장/체중': '160cm, 40kg', '학교': '구정고등학교', '특기': '피아노', '다른이름': '김미선; 김별'}
{'_id': ObjectId('61122bbaff5c0ecb95545a5c'), '배우이름': '엄정화', '흥행지수': 1959, '출연영화': ['오케이 마담', '해운대'], '랭킹': '137', '직업': '배우', '생년월일': '1969-08-17', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/umaizing/\nhttps://twitter.com/love_tangle\n', '신장/체중': '164cm, 47kg', '학교': '북원여자고등학교', '취미': '향수병 모으기, 영화보기, 책보기', '특기': '메모하기'}
{'_id': ObjectId('61122bbaff5c0ecb95545a58'), '배우이름': '김혜수', '흥행지수': 2

<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
생년월일이 없는 doc의 actor_name 만 출력하기
</div>

In [8]:
docs = actor.find({'생년월일': {'$exists':False}}, {'배우이름':1, '_id':0})
for doc in docs:
    print (doc)

{'배우이름': '이정재'}
{'배우이름': '염정아'}
{'배우이름': '김의성'}
{'배우이름': '배성우'}
{'배우이름': '이봉련'}
{'배우이름': '정만식'}
{'배우이름': '김소진'}
{'배우이름': '정우성'}
{'배우이름': '김성규'}
{'배우이름': '김병순'}
{'배우이름': '고아성'}
{'배우이름': '이솜'}


### 1.4 find의 다양한 문법 - 필드값 범위로 검색

In [9]:
docs = actor.find({'흥행지수': {'$gte': 10000}, '출연영화':'극한직업'}, {'배우이름':1, '출연영화':1, '_id':0}).sort('배우이름').limit(3)
for doc in docs:
    print(doc)

{'배우이름': '류승룡', '출연영화': ['극한직업', '명량', '광해, 왕이 된 남자', '표적', '7번방의 선물', '열혈남아']}
{'배우이름': '이하늬', '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시']}
{'배우이름': '진선규', '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨']}


### find의 다양한 문법 - or

In [10]:
docs = actor.find({'$or': [{'출연영화':'극한직업'}, {'출연영화':'더 킹'}] }, {'배우이름':1, '출연영화':1, '_id':0})
for doc in docs:
    print(doc)

{'배우이름': '진선규', '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨']}
{'배우이름': '이하늬', '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시']}
{'배우이름': '류승룡', '출연영화': ['극한직업', '명량', '광해, 왕이 된 남자', '표적', '7번방의 선물', '열혈남아']}
{'배우이름': '신하균', '출연영화': ['극한직업', '나의 특별한 형제', '박쥐', '복수는 나의 것', '미스터 주: 사라진 VIP', '지구를 지켜라!']}
{'배우이름': '이동휘', '출연영화': ['극한직업', '어린 의뢰인', '국도극장', '럭키', '새해전야', '베테랑']}
{'배우이름': '공명', '출연영화': ['극한직업', '기방도령', '도희야', '이것이 우리의 끝이다']}
{'배우이름': '오정세', '출연영화': ['극한직업', '스윙키즈', '그대 이름은 장미', '나는 나를 해고하지 않는다', '부당거래', '사이비']}
{'배우이름': '김종수', '출연영화': ['돈', '시동', '극한직업', '증인', '협상', '삼진그룹 영어토익반']}
{'배우이름': '김의성', '출연영화': ['극한직업', '창궐', '어쩌다, 결혼', '돌멩이', '더 킹', '내부자들']}
{'배우이름': '김아중', '출연영화': ['나쁜 녀석들: 더 무비', '더 킹']}
{'배우이름': '배성우', '출연영화': ['변신', '안시성', '내부자들', '지푸라기라도 잡고 싶은 짐승들', '더 킹', '특종: 량첸살인기']}
{'배우이름': '송영규', '출연영화': ['극한직업', '야구소녀']}
{'배우이름': '조인성', '출연영화': ['안시성', '모가디슈', '클래식', '더 킹']}
{'배우이름': '류준열', '출연영화': ['봉오동 전투', '독전', '뺑반', '더 킹', '택시운전사', '리틀 포레스트']}


In [11]:
docs = actor.find({ '흥행지수': {'$gte': 10000}, '$or': [{'출연영화':'극한직업'}, {'출연영화':'더 킹'}] }, {'배우이름':1, '출연영화':1, '_id':0})
for doc in docs:
    print(doc)

{'배우이름': '진선규', '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨']}
{'배우이름': '이하늬', '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시']}
{'배우이름': '류승룡', '출연영화': ['극한직업', '명량', '광해, 왕이 된 남자', '표적', '7번방의 선물', '열혈남아']}


### find의 다양한 문법 - nor
* not or

In [12]:
docs = actor.find({'$nor': [{'흥행지수': { '$gte': 10000}}, {'흥행지수': { '$lte': 2000}}]}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

{'배우이름': '김향기', '흥행지수': 9308}
{'배우이름': '신하균', '흥행지수': 9066}
{'배우이름': '이동휘', '흥행지수': 8849}


### find의 다양한 문법 - in, nin
* in: 들어가 있다.
* nin: not in - 들어가 있지 않다.

In [13]:
docs = actor.find({'흥행지수': { '$in': [9182, 8439]}}, {'배우이름':1, '흥행지수':1, '_id':0})
for doc in docs:
    print (doc)

In [14]:
docs = actor.find({'흥행지수': { '$nin': [9182, 8439]}}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

{'배우이름': '하정우', '흥행지수': 23123}
{'배우이름': '마동석', '흥행지수': 20710}
{'배우이름': '이병헌', '흥행지수': 17424}


<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
흥행지수 가 9182, 8439가 아니고, 10000 이하인 데이터를 3개만 검색하세요. (nor, in, gt 활용, 배우이름과 흥행지수만 출력)
</div>

In [15]:
docs = actor.find({'$nor': [{'흥행지수': { '$in': [9182, 8439]}}, {'흥행지수': { '$gt': 10000}}]}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)
for doc in docs:
    print (doc)

{'배우이름': '김향기', '흥행지수': 9308}
{'배우이름': '신하균', '흥행지수': 9066}
{'배우이름': '이동휘', '흥행지수': 8849}


### find의 다양한 문법 - skip, limit
* skip(n): 검색 결과 n개만큼 건너뜀
* limit(n): 검색 결과 n개만 표시

In [16]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).limit(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d4'), '배우이름': '하정우', '흥행지수': 23123, '출연영화': ['신과 함께-인과 연', '백두산', 'PMC: 더 벙커', '클로젯', '신과 함께-죄와 벌', '멋진 하루'], '랭킹': '1', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영', '다른이름': '김성훈; 河正宇'}
{'_id': ObjectId('61122bbaff5c0ecb955459d5'), '배우이름': '마동석', '흥행지수': 20710, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg', '다른이름': 'Ma Dongseok'}
{'_id': ObjectId('61122bbaff5c0ecb955459d6'), '배우이름': '이병헌', '흥행지수': 17424, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': '3', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모

In [17]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).skip(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d7'), '배우이름': '주지훈', '흥행지수': 15752, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'], '랭킹': '4', '원어명': '주지훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노', '다른이름': '주영훈'}
{'_id': ObjectId('61122bbaff5c0ecb955459d8'), '배우이름': '이성민', '흥행지수': 15560, '출연영화': ['남산의 부장들', '목격자', '공작', '비스트', '마약왕', '뺑반'], '랭킹': '5', '직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}
{'_id': ObjectId('61122bbaff5c0ecb955459d9'), '배우이름': '진선규', '흥행지수': 15508, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨'], '랭킹': '6', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}
{'_id': ObjectId('61122bbaff5c0ecb955459da'), '배우이름': '황정민', '흥행지수': 14947, '출연영화': ['공작', '인질', '다만 악에서 구하소서', '신세계', '와이키키 브라더스', '부당거래'], '랭킹': '7', '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서

In [18]:
docs = actor.find({'흥행지수': {'$gte': 10000}}).skip(3).limit(3)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d7'), '배우이름': '주지훈', '흥행지수': 15752, '출연영화': ['신과 함께-인과 연', '공작', '암수살인', '신과 함께-죄와 벌', '아수라', '좋은 친구들'], '랭킹': '4', '원어명': '주지훈', '직업': '배우', '생년월일': '1982-05-16', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/_jujihoon/\n', '신장/체중': '187cm, 68kg', '학교': '경기대학교 연기', '취미': '컴퓨터 게임', '특기': '속독, 노래, 춤, 합기도, 피아노', '다른이름': '주영훈'}
{'_id': ObjectId('61122bbaff5c0ecb955459d8'), '배우이름': '이성민', '흥행지수': 15560, '출연영화': ['남산의 부장들', '목격자', '공작', '비스트', '마약왕', '뺑반'], '랭킹': '5', '직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}
{'_id': ObjectId('61122bbaff5c0ecb955459d9'), '배우이름': '진선규', '흥행지수': 15508, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨'], '랭킹': '6', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}


### find의 다양한 문법 - list 검색

In [19]:
docs = actor.find({'출연영화': '극한직업'})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d9'), '배우이름': '진선규', '흥행지수': 15508, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨'], '랭킹': '6', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}
{'_id': ObjectId('61122bbaff5c0ecb955459dc'), '배우이름': '이하늬', '흥행지수': 12910, '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시'], '랭킹': '9', '직업': '배우', '생년월일': '1983-03-02', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/saram.leehoney\nhttps://instagram.com/honey_lee32/\n'}
{'_id': ObjectId('61122bbaff5c0ecb955459de'), '배우이름': '류승룡', '흥행지수': 11383, '출연영화': ['극한직업', '명량', '광해, 왕이 된 남자', '표적', '7번방의 선물', '열혈남아'], '랭킹': '11', '직업': '배우', '생년월일': '1970-11-29', '성별': '남', '신장/체중': '175cm, 72kg', '학교': '서울예술대학 연극', '소속사': '필름있수다', '다른이름': '류승용; 유승룡'}
{'_id': ObjectId('61122bbaff5c0ecb955459e3'), '배우이름': '신하균', '흥행지수': 9066, '출연영화': ['극한직업', '나의 특별한 형제', '박쥐', '복수는 나의 것', '미스터 주: 사라진 VIP', '지구를 지켜라!'], '랭킹': '16', '직업': '배우', '생년월일': '1974-05-30', '성별': '남', '신장/체중': '175cm, 67kg', '학교': '서울예술대학 방송연

In [20]:
docs = actor.find({'$or': [{'출연영화': '극한직업'}, {'출연영화': '사바하'}]})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459d9'), '배우이름': '진선규', '흥행지수': 15508, '출연영화': ['극한직업', '사바하', '롱 리브 더 킹: 목포 영웅', '돈', '동네사람들', '퍼펙트맨'], '랭킹': '6', '직업': '배우', '생년월일': '1977-09-13', '성별': '남'}
{'_id': ObjectId('61122bbaff5c0ecb955459dc'), '배우이름': '이하늬', '흥행지수': 12910, '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시'], '랭킹': '9', '직업': '배우', '생년월일': '1983-03-02', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/saram.leehoney\nhttps://instagram.com/honey_lee32/\n'}
{'_id': ObjectId('61122bbaff5c0ecb955459de'), '배우이름': '류승룡', '흥행지수': 11383, '출연영화': ['극한직업', '명량', '광해, 왕이 된 남자', '표적', '7번방의 선물', '열혈남아'], '랭킹': '11', '직업': '배우', '생년월일': '1970-11-29', '성별': '남', '신장/체중': '175cm, 72kg', '학교': '서울예술대학 연극', '소속사': '필름있수다', '다른이름': '류승용; 유승룡'}
{'_id': ObjectId('61122bbaff5c0ecb955459e0'), '배우이름': '박정민', '흥행지수': 10530, '출연영화': ['시동', '사바하', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '변산'], '랭킹': '13', '직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기

### find의 다양한 문법 - list 검색 (all)

In [21]:
docs = actor.find({'출연영화': { '$all': ['변산', '사바하']}})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459e0'), '배우이름': '박정민', '흥행지수': 10530, '출연영화': ['시동', '사바하', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '변산'], '랭킹': '13', '직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}


### find의 다양한 문법 - list 검색 (리스트 index 번호로 검색하기)

In [22]:
docs = actor.find({'출연영화.0': '사바하'})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459e5'), '배우이름': '이정재', '흥행지수': 8614, '출연영화': ['사바하', '다만 악에서 구하소서', '신세계', '태양은 없다', '인천상륙작전', '시월애'], '랭킹': '18', '직업': '배우', '성별': '남', '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n', '소속사': '아티스트컴퍼니'}


### find의 다양한 문법 - list 검색 (리스트 사이즈로 검색하기)

In [23]:
docs = actor.find({'출연영화': {'$size': 5}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61122bbaff5c0ecb955459dc'), '배우이름': '이하늬', '흥행지수': 12910, '출연영화': ['극한직업', '블랙머니', '연가시', '달빛궁궐', '조작된 도시'], '랭킹': '9', '직업': '배우', '생년월일': '1983-03-02', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/saram.leehoney\nhttps://instagram.com/honey_lee32/\n'}
{'_id': ObjectId('61122bbaff5c0ecb955459ec'), '배우이름': '강기영', '흥행지수': 7525, '출연영화': ['엑시트', '너의 결혼식', '가장 보통의 연애', '자산어보', '퍼즐'], '랭킹': '25', '직업': '배우', '생년월일': '1983-10-14', '성별': '남', '홈페이지': '\nhttps://instagram.com/kiyoung.kang.5/\n'}
{'_id': ObjectId('61122bbaff5c0ecb955459f1'), '배우이름': '최우식', '흥행지수': 6794, '출연영화': ['기생충', '물괴', '그대 이름은 장미', '마녀', '부산행'], '랭킹': '30', '직업': '배우', '생년월일': '1990-03-26', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/dntlrdl/\n', '신장/체중': '181cm, 65kg', '학교': '중앙대학교 아시아문화학'}
{'_id': ObjectId('61122bbaff5c0ecb95545a06'), '배우이름': '이봉련', '흥행지수': 4491, '출연영화': ['엑시트', '82년생 김지영', '마약왕', '국제수사', '생일'], '랭킹': '51', '직업': '배우', '성별': '여'}
{'_id': ObjectId('61122bbaff5c0ecb95545a0d'), '배우이

### find의 다양한 문법 (elemMatch)
* 적어도 한 개 이상의 리스트 요소가 복수 개의 조건을 동시에 만족하는 경우

In [24]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
elemmatch_sample = actor_db.sample

In [25]:
elemmatch_sample.insert_many([
    {'results': [82, 85, 88]},
    {'results': [75, 88, 91]}
])

In [26]:
docs = elemmatch_sample.find({'results': {'$gte': 90, '$lt':85}})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122d8fff5c0ecb95545a63'), 'results': [75, 88, 91]}


In [27]:
docs = elemmatch_sample.find({'results': {'$elemMatch': {'$gte':75, '$lt':80}}})
for doc in docs:
    print (doc)

{'_id': ObjectId('61122d8fff5c0ecb95545a63'), 'results': [75, 88, 91]}


<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
직업이 가수인 배우 중, 흥행지수가 가장 높은 배우순으로 10명을 출력하세요
</div>

In [28]:
docs = actor.find({'직업':'가수'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb95545a01'), '배우이름': '배수지', '흥행지수': 4814, '출연영화': ['백두산', '건축학개론'], '랭킹': '46', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': '\nhttps://twitter.com/missa_suzy\nhttps://www.instagram.com/skuukzky/\n', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트', '다른이름': '배수지;미쓰에이;'}
{'_id': ObjectId('61122bbaff5c0ecb95545a03'), '배우이름': '임윤아', '흥행지수': 4698, '출연영화': ['엑시트', '공조'], '랭킹': '48', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/yoona__lim/\n', '소속사': 'SM엔터테인먼트', '다른이름': '소녀시대; girlsgeneration; girls generation'}


<div class="alert alert-block alert-warning">
<font color="blue" size="4em">실습</font><br>
국가부도의 날에 출연한 배우를 흥행지수가 높은 순으로 10명 출력하세요 
</div>

In [29]:
docs = actor.find({'출연영화':'국가부도의 날'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print (doc)

{'_id': ObjectId('61122bbaff5c0ecb955459dd'), '배우이름': '조우진', '흥행지수': 12861, '출연영화': ['봉오동 전투', '돈', '창궐', '발신제한', '마약왕', '국가부도의 날'], '랭킹': '10', '직업': '배우', '생년월일': '1979-01-16', '성별': '남', '다른이름': '조신제'}
{'_id': ObjectId('61122bbaff5c0ecb955459e6'), '배우이름': '조한철', '흥행지수': 8446, '출연영화': ['신과 함께-인과 연', '백두산', '국가부도의 날', '세자매', '로망', '블랙머니'], '랭킹': '19', '직업': '배우', '생년월일': '1973-05-13', '성별': '남'}
{'_id': ObjectId('61122bbaff5c0ecb955459ee'), '배우이름': '유아인', '흥행지수': 7335, '출연영화': ['#살아있다', '국가부도의 날', '소리도 없이', '베테랑', '우아한 거짓말', '버닝'], '랭킹': '27', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/hongsick/\nhttps://twitter.com/seeksik\n', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영', '다른이름': '엄홍식'}
{'_id': ObjectId('61122bbaff5c0ecb955459f2'), '배우이름': '김홍파', '흥행지수': 6617, '출연영화': ['공작', '천문: 하늘에 묻는다', '국가부도의 날', '마약왕', '말모이', '괴기맨숀'], '랭킹': '31', '직업': '배우', '생년월일': '1962-01-05', '성별': '남'}
{'_id': ObjectId('